In [ ]:
from pydatafabric.vault_utils import get_secrets

aws_info = get_secrets(path="apollo/access/dev")

In [ ]:
from pydatafabric.ye import get_spark
import os

os.environ["AWS_ACCESS_KEY_ID"] = aws_info["aws_access_key_id"]
os.environ["AWS_SECRET_ACCESS_KEY"] = aws_info["aws_secret_access_key"]

spark = get_spark()

In [ ]:
partitions = spark.sql("show partitions saturn.ci_mst").collect()
dts = [r["partition"].split("=")[-1] for r in partitions]
max_dt = max(dts)

In [ ]:
from datetime import datetime, timedelta

c_dt = datetime.strptime(max_dt, '%Y%m%d')

o_dt = c_dt + timedelta(days=-1)

c_dt = datetime.strftime(c_dt, '%Y%m%d')
o_dt = datetime.strftime(o_dt, '%Y%m%d')

print('c_dt : %s' % c_dt)
print('o_dt : %s' % o_dt)

In [ ]:
spark.sql(f""" select mgmt_num  as svc_mgmt_num, ci from saturn.ci_mst where dt={c_dt} """).registerTempTable(
    'latest_table')
spark.sql(f""" select mgmt_num  as svc_mgmt_num, ci from saturn.ci_mst where dt={o_dt} """).registerTempTable(
    'old_table')

In [ ]:
query = """

    select
        l.svc_mgmt_num,
        l.ci
    from 
        latest_table l left outer join old_table o
        on l.svc_mgmt_num = o.svc_mgmt_num
    where
        o.svc_mgmt_num is null
        and l.ci is not null

"""

In [ ]:
env = ["dev", "stg", "prd"]

In [ ]:
for k in env:
    s3_path = f"s3a://datafabric-ci-mapping-prd/ci_table"
    print(s3_path)

    aws_info = get_secrets(mount_point="datafabric", path=f"aws/s3/access/prd")
    spark.conf.set("fs.s3a.access.key", aws_info["aws_access_key_id"])
    spark.conf.set("fs.s3a.secret.key", aws_info["aws_secret_access_key"])

    spark.sql(query).write.mode("overwrite").parquet(s3_path)

In [ ]:
spark.stop()